In [31]:
import pandas as pd
from sklearn.preprocessing import  MinMaxScaler
from deepctr.models import DCN
from deepctr import SingleFeat

In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
#import xgboost as xgb
from scipy.sparse import vstack, csr_matrix, save_npz, load_npz
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedKFold
import gc
gc.enable()


In [3]:
train = pd.read_csv('deepctr_train.csv')
test = pd.read_csv('deepctr_test.csv')


dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float16',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float16',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

print('Download Train and Test Data.\n')
train2 = pd.read_csv('./data/train.csv', dtype=dtypes, low_memory=True)
#test2  = pd.read_csv('./data/test.csv',  dtype=dtypes, low_memory=True)
#test2['HasDetections']=[0]*len(test2)
y_train = np.array(train2['HasDetections'])


Download Train and Test Data.



In [4]:
data=pd.concat([train,test])
sparse_features = train.columns
dense_features = sparse_features+['count']

#sparse_features = data.columns
#dense_features = []


del train
del test

print(data.shape)
for col in data.columns:
    group_by_col=data.groupby([col]).size()
    df_group_by_col = pd.DataFrame(group_by_col,columns=[col+'count'])
    df_group_by_col = df_group_by_col.reset_index()
    data = pd.merge(data,df_group_by_col,on=col,how='left')


#mms = MinMaxScaler(feature_range=(0, 1))
#data[dense_features] = mms.fit_transform(data[dense_features])

    
print(data.shape)
train = data[:8921483]
test = data[-7853253:]

del train2
#del data
gc.collect()

(16774736, 81)
(16774736, 162)


229425

In [5]:
#data=pd.concat([train,test])
print('xDeepFM\n')

# 2.count #unique features for each sparse field,and record dense feature field name
sparse_feature_list = [SingleFeat(feat, data[feat].nunique())
                        for feat in sparse_features]
dense_feature_list = [SingleFeat(feat, 0)
                        for feat in dense_features]
del data
gc.collect()
# 3.generate input data for model
test_model_input = [test[feat.name].values for feat in sparse_feature_list] + \
    [test[feat.name].values for feat in dense_feature_list]

xDeepFM



In [6]:
train.head()

,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,...,Census_FirmwareManufacturerIdentifiercount,Census_FirmwareVersionIdentifiercount,Census_IsSecureBootEnabledcount,Census_IsWIMBootEnabledcount,Census_IsVirtualDevicecount,Census_IsTouchEnabledcount,Census_IsPenCapablecount,Census_IsAlwaysOnAlwaysConnectedCapablecount,Wdft_IsGamercount,Wdft_RegionIdentifiercount
0,2,0,0,0,1,5,0,37,139,1,...,2300471,4885481,8894050,11182410,16614168,14725738,16137100,15682103,11476755,3251921
1,2,21,17,8,1,5,0,37,139,1,...,2300471,4885481,8894050,11182410,16614168,14725738,16137100,15682103,11476755,504963
2,2,0,0,0,1,5,0,37,139,1,...,5141158,12904,8894050,11182410,16614168,14725738,16137100,15682103,11476755,2549287
3,2,0,0,0,1,5,0,37,139,1,...,1789480,61193,8894050,11182410,16614168,14725738,16137100,15682103,11476755,2549287
4,2,0,0,0,1,5,0,37,139,1,...,1789480,18540,8894050,5592324,16614168,14725738,16137100,15682103,11476755,2341900


In [32]:
from sklearn.model_selection import KFold
KF=KFold(n_splits=5) 
i=0
for train_index,test_index in KF.split(train.index):
    X_train,X_valid=train.iloc[train_index],train.iloc[test_index]
    Y_train,Y_valid=y_train[train_index],y_train[test_index]

    # 4.Define Model,train,predict and evaluate
    train_model_input = [X_train[feat.name].values for feat in sparse_feature_list] + \
        [X_train[feat.name].values for feat in dense_feature_list]
    valid_model_input = [X_valid[feat.name].values for feat in sparse_feature_list] + \
        [X_valid[feat.name].values for feat in dense_feature_list]



    model = DCN({"sparse": sparse_feature_list,
                    "dense": dense_feature_list}, hidden_size=(2048,2048),embedding_size=9, final_activation='sigmoid')
    model.compile("adam", "binary_crossentropy",
                     metrics=['binary_crossentropy'], )
    history = model.fit(train_model_input, Y_train,batch_size=1024*4, epochs=4, verbose=1, validation_data=(valid_model_input,Y_valid))
    pred_ans = model.predict(test_model_input, batch_size=1024*4)
    
    if i ==0 :
         pred_ans = model.predict(test_model_input, batch_size=1024*4)
    else:
         pred_ans += model.predict(test_model_input, batch_size=1024*4)
    i = i+ 1

#submission = pd.read_csv('./data/sample_submission.csv')
#submission['HasDetections'] = lgb_test_result / counter
#submission.to_csv('lgb_submission.csv', index=False)

print('\nDone.')

Train on 7137186 samples, validate on 1784297 samples
Epoch 1/4
7137186/7137186 [==============================] - 219s 31us/step - loss: 0.6099 - binary_crossentropy: 0.6095 - val_loss: 0.6027 - val_binary_crossentropy: 0.6020
Epoch 2/4
7137186/7137186 [==============================] - 199s 28us/step - loss: 0.6007 - binary_crossentropy: 0.5997 - val_loss: 0.6003 - val_binary_crossentropy: 0.5992
Epoch 3/4
7137186/7137186 [==============================] - 199s 28us/step - loss: 0.5973 - binary_crossentropy: 0.5960 - val_loss: 0.6000 - val_binary_crossentropy: 0.5986
Epoch 4/4
7137186/7137186 [==============================] - 199s 28us/step - loss: 0.5943 - binary_crossentropy: 0.5927 - val_loss: 0.5999 - val_binary_crossentropy: 0.5982
Train on 7137186 samples, validate on 1784297 samples
Epoch 1/4
7137186/7137186 [==============================] - 222s 31us/step - loss: 0.6094 - binary_crossentropy: 0.6090 - val_loss: 0.6033 - val_binary_crossentropy: 0.6026
Epoch 2/4
7137186/7137

In [33]:
submission = pd.read_csv('./data/sample_submission.csv')
submission['HasDetections'] = pred_ans/5
submission.to_csv('DCN_Epoch_4_Fold_5_add_count.csv', index=False)

In [21]:
#原始的baseline
(0.5985 + 0.5989 + 0.5989 + 0.5985 + 0.5991)/5
#LB
#0.689

0.59878

In [22]:
#添加了count特征
#2048 2048
#1024
(0.5994 + 0.5988 + 0.5995 + 0.5994 + 0.6000)/5
#LB
#0.688

0.5994200000000001

In [26]:
#全部转为稀疏的
(0.5983 + 0.5985 + 0.5978 + 0.5981 + 0.5980)/5

0.59814

In [27]:
#epoch 4  embding 9
(0.5978 + 0.5981 + 0.5979 + 0.5981 + 0.5988)/5

0.5981400000000001

In [34]:
(0.5982 + 0.5984 + 0.5976 + 0.5984 + 0.5984)/5

0.5982